In [1]:
#Working on pyspark in Jupyterlab
import pyspark

In [2]:
from pyspark.sql import SparkSession 

In [3]:
from pyspark import SparkConf, SparkContext

In [4]:
conf = SparkConf(). \
    setMaster("local"). \
    setAppName("Orders_Revenue"). \
    set("conf.ui.port","10567")

In [5]:
sc = SparkContext(conf=conf)

22/11/21 09:44:18 WARN Utils: Your hostname, codeStation resolves to a loopback address: 127.0.1.1; using 192.168.213.83 instead (on interface wlo1)
22/11/21 09:44:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/21 09:44:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/21 09:44:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


#### Execution of the file in command prompt

spark-submit --master local --deploy-mode client --conf spark.ui.port=12901 --conf spark.dynamicAllocation.anabled=false --num-executors 1 --executor-memory 512M --executor-cores 1 file_name_for_DE.py

In [7]:
productPath = "/home/solverbot/spark-warehouse/retail_db/products/part-00000"
orderitemPath = "/home/solverbot/spark-warehouse/retail_db/order_items/part-00000"
ordersPath = "/home/solverbot/spark-warehouse/retail_db/orders/part-00000.txt"

In [8]:
products = sc.textFile(productPath)

In [9]:
type(products)

pyspark.rdd.RDD

In [14]:
help(products.saveAsTextFile)

Help on method saveAsTextFile in module pyspark.rdd:

saveAsTextFile(path: str, compressionCodecClass: Optional[str] = None) -> None method of pyspark.rdd.RDD instance
    Save this RDD as a text file, using string representations of elements.
    
    Parameters
    ----------
    path : str
        path to text file
    compressionCodecClass : str, optional
        fully qualified classname of the compression codec class
        i.e. "org.apache.hadoop.io.compress.GzipCodec" (None by default)
    
    Examples
    --------
    >>> from tempfile import NamedTemporaryFile
    >>> tempFile = NamedTemporaryFile(delete=True)
    >>> tempFile.close()
    >>> sc.parallelize(range(10)).saveAsTextFile(tempFile.name)
    >>> from fileinput import input
    >>> from glob import glob
    >>> ''.join(sorted(input(glob(tempFile.name + "/part-0000*"))))
    '0\n1\n2\n3\n4\n5\n6\n7\n8\n9\n'
    
    Empty lines are tolerated when saving to text files.
    
    >>> from tempfile import NamedTemporary

In [10]:
import getpass
username = getpass.getuser()

In [11]:
username

'solverbot'

In [62]:
readingBack = sc.textFile(f"/home/{username}/orders_and_sales")
readingBack.count()

57431

In [13]:
#The spark context is a jvm which can be used to access the JVM related classes. 
sc._gateway.jvm.org.apache

In [16]:
#The codecs for compressing the files can be located from spark context before using it
sc._gateway.jvm.org.apache.hadoop.io.compress.GzipCodec()

JavaObject id=o26

In [17]:
sc.version

'3.3.1'

In [18]:
sparksession = SparkSession.builder.appName('Read_session').getOrCreate()

In [19]:
ordersJson = sparksession.read.json("/home/solverbot/spark-warehouse/retail_db_json/orders")

In [20]:
ordersJson.take(2)

[Row(order_customer_id=11599, order_date='2013-07-25 00:00:00.0', order_id=1, order_status='CLOSED'),
 Row(order_customer_id=256, order_date='2013-07-25 00:00:00.0', order_id=2, order_status='PENDING_PAYMENT')]

In [22]:
help(ordersJson.write.json)

Help on method json in module pyspark.sql.readwriter:

json(path: str, mode: Optional[str] = None, compression: Optional[str] = None, dateFormat: Optional[str] = None, timestampFormat: Optional[str] = None, lineSep: Optional[str] = None, encoding: Optional[str] = None, ignoreNullFields: Union[bool, str, NoneType] = None) -> None method of pyspark.sql.readwriter.DataFrameWriter instance
    Saves the content of the :class:`DataFrame` in JSON format
    (`JSON Lines text format or newline-delimited JSON <http://jsonlines.org/>`_) at the
    specified path.
    
    .. versionadded:: 1.4.0
    
    Parameters
    ----------
    path : str
        the path in any Hadoop supported file system
    mode : str, optional
        specifies the behavior of the save operation when data already exists.
    
        * ``append``: Append contents of this :class:`DataFrame` to existing data.
        * ``overwrite``: Overwrite existing data.
        * ``ignore``: Silently ignore this operation if data 

In [24]:
ordersTextJson = sc.textFile('/home/solverbot/spark-warehouse/retail_db_json/orders')
ordersTextJson.take(2)

['{"order_id":1,"order_date":"2013-07-25 00:00:00.0","order_customer_id":11599,"order_status":"CLOSED"}',
 '{"order_id":2,"order_date":"2013-07-25 00:00:00.0","order_customer_id":256,"order_status":"PENDING_PAYMENT"}']

### JSON files have different load option

The json files store the data in key value pairs, and dictionaries. So when the files are read from Json text files, the additional step of converting the individual rows to dictionaries has to be completed using the Json.loads() option as shown below

In [39]:
import json
json.loads(ordersTextJson.take(1)[0])["order_status"]

'CLOSED'

In [40]:
ordersJsonDict = ordersTextJson.map(lambda d: json.loads(d))

In [41]:
ordersTextJson.take(1)

['{"order_id":1,"order_date":"2013-07-25 00:00:00.0","order_customer_id":11599,"order_status":"CLOSED"}']

In [45]:
for i in ordersJsonDict.take(5): print(i['order_status'])

CLOSED
PENDING_PAYMENT
COMPLETE
CLOSED
COMPLETE


### Some of the map options are very interesting

In [46]:
orders = sc.textFile(ordersPath)
orders.take(2)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT']

In [50]:
orderItems = sc.textFile(orderitemPath)
orderItems.take(2)

['1,1,957,1,299.98,299.98', '2,2,1073,1,199.99,199.99']

In [47]:
#Notice the "check" expression is against the first element of the split 
orders.filter(lambda order: "2013-08" in order.split(',')[1]).take(2)

['1297,2013-08-01 00:00:00.0,11607,COMPLETE',
 '1298,2013-08-01 00:00:00.0,5105,CLOSED']

In [56]:
#Notice the "startswith" expression is against the first element of the split 
ordersFiltered = orders.filter(lambda order: order.split(',')[1].startswith("2013-08")). \
                    map(lambda item: (int(item.split(',')[0]), 1))
ordersFiltered.take(5)

[(1297, 1), (1298, 1), (1299, 1), (1300, 1), (1301, 1)]

In [61]:
ordersFiltered.count()

5680

In [53]:
orderItemsFiltered = orderItems.map(lambda order: (int(order.split(',')[1]), 
                                                   (int(order.split(',')[2]), float(order.split(',')[4]))))
    
orderItemsFiltered.take(2)

[(1, (957, 299.98)), (2, (1073, 199.99))]

In [58]:
joinOrders = orderItemsFiltered.join(ordersFiltered)
joinOrders.take(2)

[(1300, ((191, 199.98), 1)), (1300, ((1014, 199.92), 1))]

In [59]:
for i in joinOrders.take(10): print(i)

(1300, ((191, 199.98), 1))
(1300, ((1014, 199.92), 1))
(1302, ((365, 59.99), 1))
(1302, ((502, 100.0), 1))
(1302, ((365, 239.96), 1))
(1306, ((191, 399.96), 1))
(1306, ((191, 299.97), 1))
(1306, ((1004, 399.98), 1))
(1308, ((1073, 199.99), 1))
(1308, ((1073, 199.99), 1))


In [60]:
joinOrders.count()

14268

In [72]:
revenueByPid=joinOrders. \
                map(lambda rec: rec[1][0]). \
                reduceByKey(lambda tot, ele: round(tot + ele, 1))

In [73]:
revenueByPid.take(5)

[(1014, 247251.2),
 (502, 261150.0),
 (1004, 562400.0),
 (804, 2039.9),
 (172, 2340.0)]

In [63]:
#The file in the local file system can be read, lines can be split and the parallelized
productslocal = open(productPath).read().splitlines()
productsRdd = sc.parallelize(productslocal)
productsRdd.take(2)

['1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy',
 "2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat"]

In [62]:
products = sc.textFile(productPath)
products.take(2)

['1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy',
 "2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat"]

In [76]:
productRevenue = products.map(lambda d: (int(d.split(',')[0]), d.split(',')[2])) \
        .join(revenueByPid)

In [78]:
productOutput = productRevenue.map(lambda p: f'{p[1][0]} => {str(p[1][1])}')
productOutput.take(2)

['Elevation Training Mask 2.0 => 1999.8',
 "Nike Kids' Grade School KD VI Basketball Shoe => 1099.9"]

In [79]:
ordersCount = sc.accumulator(0)

In [82]:
def getOrderTuples(rec):
    ordersCount.add(1)
    return (int(rec.split(',')[0]), 1)

In [83]:
ordersFilteredAccum = orders.filter(lambda order: order.split(',')[1].startswith("2013-08")). \
                    map(lambda item: getOrderTuples(item))
ordersFilteredAccum.take(5)

[(1297, 1), (1298, 1), (1299, 1), (1300, 1), (1301, 1)]

In [84]:
print(ordersCount)

5


In [85]:
ordersFilteredData = ordersFilteredAccum.collect()

In [86]:
ordersCount #The earlier action was stored in the accumulator. 

Accumulator<id=0, value=5685>

In [90]:
joinOrdersAccum = orderItemsFiltered.join(ordersFilteredAccum)

In [92]:
joinOrdersAccum.take(2)

[(1300, ((191, 199.98), 1)), (1300, ((1014, 199.92), 1))]

In [93]:
ordersCount

Accumulator<id=0, value=11365>

In [95]:
#Understanding broadcasters

productsDict = dict(map(lambda p: (int(p.split(',')[0]), p.split(',')[1]), productslocal))

In [97]:
bv = sc.broadcast(productsDict)

In [102]:
productRevenue.take(1)

[(24, ('Elevation Training Mask 2.0', 1999.8))]

In [105]:
revenueByPid.map(lambda pdtRev: bv.value[pdtRev[0]]).take(2)

['46', '24']

In [106]:
revenueByPdtIdBct = revenueByPid.map(lambda pdtRev: bv.value[pdtRev[0]])

In [108]:
revenueByPdtIdBct.take(2)

['46', '24']

In [109]:
ordersPart = sc.textFile(ordersPath, 4)

In [110]:
ordersPart.count()

68883

In [111]:
orderItemsPart = sc.textFile(orderitemPath,8)

In [115]:
orderItemsPart.count()

172198

In [116]:
orderItemsPartMap = orderItemsPart.map(lambda x: (int(x.split(',')[0]),x))
ordersPartMap = ordersPart.map(lambda y: (int(y.split(',')[0]), y))

In [117]:
orderItemsPartMap.count()

172198

In [118]:
ordersPartMap.count()

68883

In [128]:
#The number of tasks the item creates can be controlled using the partitions
orderItemsPart = sc.textFile(orderitemPath,8)

In [129]:
orderItemsPartMap = orderItemsPart.map(lambda x: (int(x.split(',')[1]),x))
ordersPartMap = ordersPart.map(lambda y: (int(y.split(',')[0]), y))

In [130]:
orderItemsPartMap.count()

172198

The number of tasks created for the join will be 12, since order file is 4 parts and orderitems file is 12 part

In [131]:
orderItemsPartMap.join(ordersPartMap).count()

172198

In [132]:
#This will create 2 files as output
orderItemsPartMap.coalesce(2).saveAsTextFile('ordersCoalesced')

In [133]:
#This will create more files, 8#
orderItemsPartMap.saveAsTextFile("ordersUncoalesced")